# Quantitative Life Science

## Specie coexistence in a neutral dynamics with enviormental noise

Michele Puppin - 1227474

# Simulations - Well-mixed population 

In [ ]:
# Import packages
import random
import pandas as pd
import numpy as np 
from joblib import Parallel, delayed
from numba import jit
import time
import math
import csv
import os

## 1) Linear fitness case

In [ ]:
@jit(nopython=True) # Ddecorator for optimization by Numba’s JIT compiler
def gillespie_wm_lfc(N, lambdaAB, k, sigma): # Gillespie algorithm
    
    x = random.random()
    if x < 0.5: # Initialize state of the environment with uniform probability
        epsilon = -1.0
    else: 
        epsilon = 1.0
        
    t = 0.0
    nA = int(N/2) # Initial number of individuals of species A
    
    while nA != N and nA != 0: # Run until species A monodominates or gets extinct
        
        r1 = random.random()
        r2 = random.random()
        
        lambdaA = lambdaAB + sigma * epsilon / 2.0 # Compute the fitness based on state of the environment
        lambdaB = lambdaAB - sigma * epsilon / 2.0
        
        rAB = lambdaA * nA * (N - nA) / N # Compute reaction rate
        rBA = lambdaB * nA * (N - nA) / N
        r_tot = rAB + rBA + k # Compute total rate
        
        dt = np.log(1.0/r1) / r_tot # Extract the time of the reaction from an exponential distribution
        t += dt
        
        if r2 < k / r_tot: 
            epsilon = epsilon*(-1.0) # Cheange the environment with probability k / r_tot
        else:
            if r2 < (k + rAB) / (r_tot): 
                nA = nA - 1 # nA decreases with probability rAB / (r_tot)
            else: 
                nA = nA + 1 # nA increases with probability rBA / (r_tot)
        
    return t

In [ ]:
def ext_mean_time_wm_lfc(N, tau, niterations): # Run Gillespie algorithm for different population size

    lambdaAB = 0.5 # Initialize fitness
    k = 1.0 / (2.0 * tau) # Compute the rate of the state of the environmental 
    sigma = 0.25 # Initialize the variability of the environment

    filename = "wm_lfc_tau"+str(tau)+"sigma"+str(sigma)+".txt"

    if os.path.exists(filename): 
        os.remove(filename)

    for i in range(len(N)): # Run Gillespie algorithm for each population size
        print("N: ", N[i], "n_iter: ", niterations[i])
        # Run Gillespie algorithm in parallel for niterations realizations
        res = Parallel(n_jobs=-1, verbose=1)(delayed(gillespie_wm_lfc)(N[i], lambdaAB, k, sigma) for it_er in range(niterations[i]))

        with open(filename, "a") as f: # Store mean values of extintion time between realizations
            writer = csv.writer(f)
            writer.writerow((N[i], np.mean(res))) 

In [ ]:
niterations = [100000, 50000, 25000, 12500, 6000, 3000, 2000, 1000, 1000, 1000] # Number of realizations for each population
N = [math.floor(i/2)*2 for i in np.logspace(2,6,10)] # List of population size
tau_list = [1.0/16.0 , 1.0/8.0, 1.0/4.0, 1.0/2.0, 1.0, 2.0, 4.0] # List of correlation time values

for tau in tau_list: # Run for different values of correlation time
    print("Tau: ", tau) 
    ext_mean_time_wm_lfc(N, tau, niterations)

## 2) Relative fitness case

In [ ]:
@jit(nopython=True) # Ddecorator for optimization by Numba’s JIT compiler
def voter_model_wm_rfc(N, sAB, k, sigma): # Voter model dynamics
    
    x = random.random() # Initialize state of the environment with uniform probability
    if x < 0.5:
        epsilon = -1.0
    else: 
        epsilon = 1.0
        
    t = 0.0
    nA = int(N/2) # Initial number of individuals of species A
    
    print(tau)
    
    while nA != N and nA != 0: # Run until species A monodominates or gets extinct
        
        r1 = random.random()
        r2 = random.random()
        
        sA = sAB + sigma * epsilon / 2.0 # Compute the number of seeds 
        sB = sAB - sigma * epsilon / 2.0
        
        lambdaA = sA / (sA * nA + sB * (N - nA)) # Compute the fitness based on the number of seeds
        lambdaB = sB / (sA * nA + sB * (N - nA))
        
        rAB = lambdaA * nA * (N - nA) / N # Compute the probabilities
        rBA = lambdaB * nA * (N - nA) / N 
        
        if r1 < rAB: 
            nA = nA + 1 # nA increases with probability rAB
        else: 
            if r1 < (rAB + rBA):
                nA = nA - 1 # nA decreases with probability rAB
                
        dt = 1.0 / N # Time step
        t += dt
        
        if r2 < (1.0 - np.exp(-k*dt)):
            epsilon = epsilon*(-1.0) # Cheange the environment with probability given by rate k
            
    return t

In [ ]:
def ext_mean_time_wm_rfc(N, tau, niterations): # Run Voter model dynamics for different population size

    sAB = 0.5 # Initialize the number of seed
    k = 1.0 / (2.0 * tau) # Compute the rate of the state of the environmental 
    sigma = 0.25 # Initialize the variability of the environment

    filename = "wm_rfc_tau"+str(tau)+".txt"

    if os.path.exists(filename): 
        os.remove(filename)

    for i in range(len(N)): # Run Voter model dynamics for each population size
        print("N: ", N[i], "n_iter: ", niterations[i])
        # Run Voter model dynamics in parallel for niterations realizations
        res = Parallel(n_jobs=-1, verbose=1)(delayed(voter_model_wm_rfc)(N[i], sAB, k, sigma) for it_er in range(niterations[i]))

        with open(filename, "a") as f: # Store mean values of extintion time between realizations
            writer = csv.writer(f)
            writer.writerow((N[i], np.mean(res)))

In [ ]:
niterations = [100, 100, 100, 100, 100, 100, 100, 100, 100, 100] # Number of realizations for each population
N = [math.floor(i/2)*2 for i in np.logspace(2,6,10)] # List of population size
tau_list = [1.0/16.0 , 1.0/8.0, 1.0/4.0, 1.0/2.0, 1.0, 2.0, 4.0] # List of correlation time values

for tau in tau_list: # Run for different values of correlation time
    print("Tau: ", tau) 
    ext_mean_time_wm_rfc(N, tau, niterations)